In [1]:
import numpy as np
from itertools import combinations
import collections
import pandas as pd
import cv2
import time
from datetime import timedelta

In [2]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [3]:
c_num = 3
num_users = 10
noniid_c = 2
classes, images = c_num, int(30/c_num)
classes_indx = [i for i in range(classes)]
users_dict = {i: [] for i in range(num_users)}
indeces = np.arange(classes*images)
unsorted_labels = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2]
print("In covid non IID: len unsorted labels = ", len(unsorted_labels))
print("In covid non IID: unsorted labels = ", unsorted_labels)

indeces_unsortedlabels = np.vstack((indeces, unsorted_labels))
shuffled_indices = np.random.permutation(len(indeces_unsortedlabels[0]))
print("In covid non IID: shuffled indices = ", shuffled_indices)
indeces_unsortedlabels[0] = indeces_unsortedlabels[0][shuffled_indices]
indeces_unsortedlabels[1] = indeces_unsortedlabels[1][shuffled_indices]
print("In covid non IID: indeces_unsortedlabels ", indeces_unsortedlabels)
indeces_labels = indeces_unsortedlabels[:, indeces_unsortedlabels[1, :].argsort()]
print("In covid non IID: indeces_labels ", indeces_labels)
indeces = indeces_labels[0, :]
indeces_labels.astype(int)
indeces.astype(int)
#     print(indeces_labels)
    
# label list with index
index_label = [[] for i in range(c_num)]
print("index_label: ", index_label)
for i in range(len(indeces_labels[1])):
    index_label[indeces_labels[1][i]].append(indeces_labels[0][i])
    print("index_label: ", index_label)
        
client_classes = []
comb = []
for i in list(combinations(list(range(0,c_num)), noniid_c)):
    print(i)
    comb.append(i)
print("comb ", comb)

# classes of client
for i in range(num_users):
    client_classes.append(comb[i%c_num])
client_classes = np.array(client_classes)
c = client_classes.flatten()
print("client_classes ", client_classes)

# count of labels
label_count = collections.Counter(c)
print("label count ", label_count)

for i in range(len(label_count)):
    index_label[i] = split(index_label[i], label_count[i])
    index_label[i] = list(index_label[i])
    
temp = []
#     users_dict = dict.fromkeys(range(10), [])
print("users_dict ", users_dict)
for i in range(len(client_classes)):
    for j in range(len(client_classes[i])):
        cur_cls = client_classes[i][j]
        temp = index_label[cur_cls].pop()
#         print("temp ", temp)
        users_dict[i] = np.concatenate((users_dict[i], np.array(temp)), axis=0).astype(int)

for i in range(len(users_dict)):
    users_dict[i] = set(users_dict[i])

print("users_dict: ", users_dict)

In covid non IID: len unsorted labels =  30
In covid non IID: unsorted labels =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
In covid non IID: shuffled indices =  [ 7 11 13 27 12 26 19  3 24 10  5  4 20  8 25  9 18 16 17  2  1  6 29  0
 28 14 15 21 23 22]
In covid non IID: indeces_unsortedlabels  [[ 7 11 13 27 12 26 19  3 24 10  5  4 20  8 25  9 18 16 17  2  1  6 29  0
  28 14 15 21 23 22]
 [ 0  1  1  2  1  2  1  0  2  1  0  0  2  0  2  0  1  1  1  0  0  0  2  0
   2  1  1  2  2  2]]
In covid non IID: indeces_labels  [[ 7  2  9  0  8  4  1  5  3  6 14 10 19 15 12 13 18 16 17 11 28 21 25 23
  20 24 26 27 29 22]
 [ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2
   2  2  2  2  2  2]]
index_label:  [[], [], []]
index_label:  [[7], [], []]
index_label:  [[7, 2], [], []]
index_label:  [[7, 2, 9], [], []]
index_label:  [[7, 2, 9, 0], [], []]
index_label:  [[7, 2, 9, 0, 8], [], []]
index_label:  [[7, 2, 9, 0, 8, 4], [], []]

In [4]:
start = time.time()
data_info = pd.read_csv('./train.csv', header=None)
print("In covidDataset init len data_info = ", len(data_info))
img = []
lb = []
for i in range(len(data_info)):
    img_name = data_info.iloc[i, 0]
    image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (32,32))
    label = data_info.iloc[i, 1]
    label = np.array([label])
    img.append(image)
    lb.append(label)
print("self.image shape", len(img))
print("self.label shape", len(lb))


In covidDataset init len data_info =  3012
self.image shape 3012
self.label shape 3012


In [5]:
image = np.asarray(img)
print(image.shape)

(3012, 32, 32)


In [7]:
lb = np.asarray(lb)
print(lb.shape)

(3012, 1)


In [6]:
read_time = round(time.time()-start)
print("* [read_time] = ", str(timedelta(seconds=(read_time))))

* [read_time] =  0:00:18
